In [9]:
import os
import sys
import cv2
import torch

In [10]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/data')))
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/utils')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/vision')))

In [22]:
# import cv_utils
# import file_utils as futils
# import data_loader as dload
# import vision
from vision.engine import train_one_epoch, evaluate
from vision.v_utils import collate_fn

In [12]:
classes_list = ["Drone", "Background"]

In [13]:
torch.manual_seed(42)

In [82]:
class YoloDroneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a Drone Yolo dataset.
    
    Args:
        split: train, test or val
        transforms (None): a list of PyTorch transforms to apply to images and targets when loading
    """

    def __init__(
        self,
        dataset_path='.',
        split='test',
        transforms=None,
        classes=classes_list,
    ):
        self.split = split
        self.transforms = transforms
        self.classes = classes

        self.images_path = []
        self.labels_path = []

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

        images_folder = os.path.join(dataset_path, 'images',split)
        labels_folder = os.path.join(dataset_path, 'labels', split)
        for image in os.listdir(images_folder):
          img_path = os.path.join(images_folder, image)
        
          label = image.split('.')[0] + '.txt'
          label_path = os.path.join(labels_folder, label)
          with open(label_path, 'r') as f:
            label_lines = f.readlines()
          if len(label_lines) != 0:   
            self.labels_path.append(label_path)
            self.images_path.append(img_path)

    def __getitem__(self, idx):
        # reading the images and converting them to correct color  
        img_path = self.images_path[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # # prepairing target
        label_path = self.labels_path[idx] 
        with open(label_path, 'r') as f:
            label_lines = f.readlines() 
        
        # cv2 image gives size as height x width    
        wt = img.shape[1]
        ht = img.shape[0]

        boxes = []
        labels = []

        # detections = sample[self.gt_field].detections
        for label in label_lines:
            class_id, x_center, y_center, bbox_width, bbox_height = map(float, label.split())
            if x_center > 1 or y_center > 1 or  bbox_width > 1 or  bbox_height > 1:
                print("label_path:",label_path)
            boxes.append([x_center * wt, y_center * ht, (x_center + bbox_width) * wt, (x_center + bbox_height) * ht])
            labels.append(1)  # drone class

        
        # applying augmentations
        if self.transforms is not None:
            transformed = self.transforms(image=img,bboxes=boxes, category_ids=labels)
            img = transformed["image"]
            boxes = transformed["bboxes"]
            labels = transformed["category_ids"]

        # convert boxes into a torch.Tensor                
        boxes = torch.as_tensor(boxes, dtype=torch.float32)                
            
        target = {}
        target["boxes"] = boxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])

        if len(boxes) != 0:
            # getting the areas of the boxes
            target["area"] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd
        target["iscrowd"] = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        return img, target

    def __len__(self):
        return len(self.images_path)

    def get_classes(self):
        return self.classes
    


In [27]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [88]:
train_transform = A.Compose(
    [
        A.LongestMaxSize(320),
        # A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        A.RandomSizedBBoxSafeCrop(width=300, height=300, erosion_rate=0.1),
     
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),
        A.ToFloat(max_value=255, p=1, always_apply=True),

        ToTensorV2(p=1.0)
    ],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
)

test_transform = A.Compose(
    [
        A.LongestMaxSize(300),
        # A.PadIfNeeded(min_height=300, min_width=300, border_mode=0),
        A.ToFloat(max_value=255, p=1, always_apply=True),

        ToTensorV2(p=1.0)
    ],
    bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [89]:
yolo_train_dataset = YoloDroneTorchDataset('../dataset/yolo', 'train', test_transform) 
yolo_test_dataset = YoloDroneTorchDataset('../dataset/yolo', 'test', test_transform)
yolo_val_dataset = YoloDroneTorchDataset('../dataset/yolo', 'val')

In [85]:
print(len(yolo_train_dataset), len(yolo_test_dataset), len(yolo_val_dataset))

12630 2706 2708


In [17]:
import torchvision
model = torchvision.models.detection.ssd300_vgg16(pretrained=True)

/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
from torchvision.models.detection.ssd import SSDHead

head = SSDHead(in_channels=[512, 1024, 512, 256, 256, 256] , num_anchors=[4,6,6,6,4,4] , num_classes=2)
model.head = head

In [19]:
bs = 2
test_bs = 1
num_epochs = 10
learning_rate = 0.0005

In [90]:
data_loader = torch.utils.data.DataLoader(
    yolo_train_dataset, batch_size=bs, shuffle=True, num_workers=2,
    collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    yolo_test_dataset, batch_size=test_bs, shuffle=False, num_workers=2,
    collate_fn=collate_fn)

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device %s" % device)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=learning_rate,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

Using device cuda


In [91]:
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

/tmp/ipykernel_15013/559415089.py:80: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  target["labels"] = torch.as_tensor(labels, dtype=torch.int64)


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_15013/559415089.py", line 70, in __getitem__
    transformed = self.transforms(image=img,bboxes=boxes, category_ids=labels)
  File "/home/jacksonrr3/hse/hse_dl_project/.venv/lib/python3.8/site-packages/albumentations/core/composition.py", line 346, in __call__
    s